<a href="https://colab.research.google.com/github/Yiming-xie-2021/aiperson_utility/blob/main/easydataset%E6%95%B0%E6%8D%AE%E5%A4%84%E7%90%86.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import json
import re

SYSTEM_PROMPT = """你是一名地产地图直播领域的专家，长期服务于用户购房决策，擅长根据给定的【背景资料】回答用户提出的问题：
- 从用户的问题和背景资料识别购房者真实意图与隐性焦虑
- 提炼这类购房人群的共性问题
- 根据背景资料用专业但不说教的方式给出能改变认知层级的答案
你的目标不是简单“答题”，而是帮助一类购房用户意识到认知盲区、规避决策风险，并引导其进一步咨询。"""

# ===================== 核心解析与检测逻辑 =====================

def _build_patterns():
    """
    构造仅基于「思考链」和「答案正文」的关键词正则，
    允许前后存在 #、*、空格、全角/半角冒号等格式差异。
    """
    # 匹配类似：
    # ### 思考链
    # ## 思考链：
    # **思考链：**
    # 思考链:
    # 思考链 ：
    think_pattern = r"[#\*\s]*思考链[\s：:：]*"
    answer_pattern = r"[#\*\s]*答案正文[\s：:：]*"
    return think_pattern, answer_pattern


THINK_PATTERN, ANSWER_PATTERN = _build_patterns()


def detect_format_errors(text):
    """
    检测「思考链 / 答案正文」格式问题：
    - 只要能在文本中找到思考链和答案正文两段的锚点，就认为格式正常
    - 找不到任意一个，就算格式有问题
    """
    errors = []

    think_match = re.search(THINK_PATTERN, text)
    answer_match = re.search(ANSWER_PATTERN, text)

    if not think_match:
        errors.append("无法识别思考链部分（未匹配到『思考链』相关标题）")

    if not answer_match:
        errors.append("无法识别答案正文部分（未匹配到『答案正文』相关标题）")

    return errors


def parse_output(output_text):
    """
    从 output 中提取思考链与答案正文：
    - 只基于「思考链」和「答案正文」两个关键词
    - 兼容多种标题写法（##、###、**思考链：**、思考链: 等）
    - 若任一部分找不到，则：
        - think 为空字符串
        - answer 为原文本整体（保证不丢内容）
    """
    think_match = re.search(THINK_PATTERN, output_text)
    answer_match = re.search(ANSWER_PATTERN, output_text)

    # 如果任意一个不存在，认为解析失败，返回空 think + 全文 answer
    if not think_match or not answer_match:
        return "", output_text.strip()

    think_start = think_match.start()
    answer_start = answer_match.start()

    # 保证按顺序切分：先出现的是思考链，后出现的是答案正文
    if think_start < answer_start:
        think_block = output_text[think_start:answer_start]
        answer_block = output_text[answer_start:]
    else:
        # 极端情况下答案正文在前，也兼容处理
        answer_block = output_text[answer_start:think_start]
        think_block = output_text[think_start:]

    # 去掉各自标题本身，只保留内容
    think = re.sub(THINK_PATTERN, "", think_block).strip()
    answer = re.sub(ANSWER_PATTERN, "", answer_block).strip()

    return think, answer

# ===================== 主转换逻辑 =====================

def convert_to_sharegpt(input_file, output_file, error_file):
    """
    将原始 JSONL 转换为 Qwen/ms-swift 训练用 JSONL：
    - messages: system / user / assistant
    - assistant.content 使用 <think>...</think> + 答案正文
    - 无思考链时，在 user 末尾添加 " /no_think"
    - 思考链 / 答案正文无法识别的样本，额外写入 error_file 方便人工检查
    """

    count_with_think = 0
    count_without_think = 0
    count_format_errors = 0

    with open(input_file, "r", encoding="utf-8") as f_in, \
         open(output_file, "w", encoding="utf-8") as f_out, \
         open(error_file, "w", encoding="utf-8") as f_err:

        for idx, line in enumerate(f_in, start=1):
            if not line.strip():
                continue

            obj = json.loads(line)

            instruction = obj.get("instruction", "")
            chunk = obj.get("chunk", "")
            output_text = obj.get("output", "")

            # ====== 检测格式问题（仅基于「思考链 / 答案正文」） ======
            fmt_errors = detect_format_errors(output_text)
            if fmt_errors:
                count_format_errors += 1

                # 控制台预览
                print(f"\n====== 第 {idx} 条数据存在格式问题 ======")
                print("格式错误：", fmt_errors)
                print("原始 output 片段：")
                print(output_text[:300], "...\n")

                # 写入错误数据文件，方便你单独排查
                error_obj = {
                    "index": idx,
                    "errors": fmt_errors,
                    "raw_output": output_text,
                    "instruction": instruction,
                    "chunk": chunk
                }
                f_err.write(json.dumps(error_obj, ensure_ascii=False) + "\n")

            # ====== 正常转换流程 ======

            # user = instruction + 背景资料
            user_content = f"{instruction}\n\n【背景资料】{chunk}"

            # 提取 think / answer
            think, answer = parse_output(output_text)
            has_think = len(think.strip()) > 0

            if has_think:
                count_with_think += 1
            else:
                count_without_think += 1
                # 选项二：无思考链时，在 query 末尾加入 /no_think
                user_content += " /no_think"

            # 构造 assistant 内容（符合 ms-swift 带 think 的格式）
            assistant_content = f"<think>\n{think}\n</think>\n\n{answer}"

            sharegpt_obj = {
                "messages": [
                    {"role": "system", "content": SYSTEM_PROMPT},
                    {"role": "user", "content": user_content},
                    {"role": "assistant", "content": assistant_content}
                ]
            }

            f_out.write(json.dumps(sharegpt_obj, ensure_ascii=False) + "\n")

    # 统计信息
    print("\n====== 数据统计 ======")
    print(f"有思考链（think）的数据：{count_with_think}")
    print(f"无思考链（no think）的数据：{count_without_think}")
    print(f"思考链/答案正文无法识别（格式问题）的数据：{count_format_errors}")
    print("======================\n")


# ===================== 调用示例 =====================

convert_to_sharegpt(
    "/content/datasets-WBKsXj0I5CgG-custom-2025-12-19.jsonl",
    "sharegpt_output.jsonl",
    "format_errors.jsonl"
)



====== 数据统计 ======
有思考链（think）的数据：354
无思考链（no think）的数据：0
思考链/答案正文无法识别（格式问题）的数据：0

